# Exam Preparation Deep Learning
18.03.2022, Thomas Iten

**Content**
1. Dense Network
2. AutoEncoder
3. CNN

## 1. Dense Network

### 1.1 Initialization

In [25]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

#
# Setup device (cuda / cpu)
#
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
print("Device:")
print(device)

Device:
cpu


### 1.2 Download data

In [26]:
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor())                       # ToTensor() erstellt proprietäres Format für PyTorch

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor())

### 1.3 Explore data

In [27]:
print("#")
print("# Train Data")
print("#")
print(train_data)

print("\n#")
print("# Train Data Classes")
print("#")
print(train_data.classes)
print()

print("\n#")
print("# Train Data ClassToIndex")
print("#")
print(train_data.class_to_idx)

#
# Train Data
#
Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

#
# Train Data Classes
#
['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


#
# Train Data ClassToIndex
#
{'T-shirt/top': 0, 'Trouser': 1, 'Pullover': 2, 'Dress': 3, 'Coat': 4, 'Sandal': 5, 'Shirt': 6, 'Sneaker': 7, 'Bag': 8, 'Ankle boot': 9}


### 1.4 Create Data Loader

In [28]:
batch_size = 64
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader  = DataLoader(test_data, batch_size=batch_size, shuffle=True)

for X, y in train_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


### 1.5 Neuronal Network

In [29]:
class NeuralNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(     # Einfaches lineares/sequentielles Netzwerk, 2 Layer
            nn.Flatten(),
            nn.Linear(28*28, 128),                  # Input=28x28 / Output=128
            nn.ReLU(),
            nn.Linear(128,10),                      # Input=128   / Output = 10
            nn.Softmax())

    def forward(self, x):
        x = self.linear_relu_stack(x)
        return x


model = NeuralNetwork().to(device)                  # Angabe wo das ausgeführt werden soll
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=10, bias=True)
    (4): Softmax(dim=None)
  )
)


### 1.6 Loss und Optimizer

In [30]:
# Backward
# - Definition Backward Loss Funktion
# - Build partial derivates of loss function and grads
loss_fn = nn.CrossEntropyLoss() # MSE

# Step
# - Optimizer aktualisiert die Gewichte
# - Der optimizer macht den Update step intelligenter
# - Update: params = params - learing_rate * grad
# - Besser ist Adam optimizer
#   optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

### 1.7 Train und Test Functions

In [31]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):     # Mit enumerate erhält man batch nummer
        X, y = X.to(device), y.to(device)           # je Bacht Bilddaten in X, Label in Y

        #
        # Compute prediction error
        #
        pred = model(X)                             # Impliziter Aufruf von model.forward
        loss = loss_fn(pred, y)

        #
        # Backpropagation
        #
        optimizer.zero_grad()                       # Reset Anfangswerte für jeden Batch

        loss.backward()                             # X.size Ableitungen und Werte
                                                    # 1. Partielle Ableitungen bilden
                                                    # 2. Werte werden in Ableitungsformel ein geplugged
                                                    #    X Werte, Y Werte, Gewichte und Bias
                                                    #    Pro Ableitung erhlät man einen Skalar (m_deriv)
                                                    #    Ein Gradient pro Feature und Bias

        optimizer.step()                            # Update der Gradienten (konkrete Werte der Ableitung)
                                                    # Gewichte/Bias = Gewichte/Bias bisher - LR * Gradienten

        #
        # Zwischenresultate zeigen
        #
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X) # batch * len(X) = Position Build
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():                           # no_grad: keine Aufzeichnung, nur Prediction
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()    # item() holt den Wert aus dem Objekt

                                                    # argmax holt grösste von den 10 Kategorien
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

### 1.8 Training

In [32]:
batch_size = 64
epochs = 10

train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader  = DataLoader(test_data,  batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    print("Epoch:", epoch+1, "\n-------------------------------")
    train(dataloader=train_dataloader, model=model, loss_fn=loss_fn, optimizer=optimizer)
    test( dataloader=test_dataloader,  model=model, loss_fn=loss_fn)
    # train(train_dataloader, model, loss_fn, optimizer)
    # test (test_dataloader,  model, loss_fn)

print("Done!")

Epoch: 1 
-------------------------------
loss: 2.300862  [    0/60000]
loss: 1.825657  [ 6400/60000]
loss: 1.655431  [12800/60000]
loss: 1.753543  [19200/60000]
loss: 1.698539  [25600/60000]
loss: 1.667875  [32000/60000]
loss: 1.641326  [38400/60000]
loss: 1.669391  [44800/60000]
loss: 1.633124  [51200/60000]
loss: 1.666724  [57600/60000]
Test Error: 
 Accuracy: 82.7%, Avg loss: 1.639786 

Epoch: 2 
-------------------------------
loss: 1.599909  [    0/60000]
loss: 1.635655  [ 6400/60000]
loss: 1.604491  [12800/60000]
loss: 1.668910  [19200/60000]
loss: 1.651608  [25600/60000]
loss: 1.663676  [32000/60000]
loss: 1.590811  [38400/60000]
loss: 1.657717  [44800/60000]
loss: 1.634533  [51200/60000]
loss: 1.644574  [57600/60000]
Test Error: 
 Accuracy: 83.8%, Avg loss: 1.627879 

Epoch: 3 
-------------------------------
loss: 1.590475  [    0/60000]
loss: 1.615870  [ 6400/60000]
loss: 1.590615  [12800/60000]
loss: 1.656596  [19200/60000]
loss: 1.629151  [25600/60000]
loss: 1.633142  [320

## 2. AutoEncoder
